# Imports

In [1]:
## imports 
import pandas as pd
import numpy as np
import yaml
import plotnine
from plotnine import *

import bitdotio

## print mult
## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## function to feed path name to load
## credentials
def load_creds(path: str):
    with open(path, 'r') as stream:
        try:
            creds = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return(creds)


# 0. Preliminary: define connection and read sample of data

Task: 

- Connect to the database - instructions here and API key on Canvas: https://github.com/rebeccajohnson88/PPOL564_slides_activities/issues/65
- Read a sample of data (first five rows) from the `caseinit` table in the database

In [2]:
creds = load_creds("../../cred.yml")
b = bitdotio.bitdotio(creds['class_database']['api_key'])
cnx = b.get_connection("rebeccajohnson88/ppol564_classdb")

# 1. Row and column filtering

## 1.1 Pulling columns

*Task*: pull the `CASE_ID` and `CASE_PARTICIPANT_ID` columns from the `caseinit` table

## 1.2 Subsetting rows



### 1.2.1 Subsetting rows based on logical conditions

*Task*: pull those two columns and restrict to rows where `AGE_AT_INCIDENT` is older than 40

*Task*: pull those two columns and restrict to rows where `RACE` is Black or HISPANIC 

### 1.2.2 Subsetting rows based on similarity to a string

*Task*: use the `like` command to subset to rows with Black anywhere in the `RACE` column

## 1.3 Creating new columns

*Task*: create a new column-- `charge_update`-- that takes the value "Same offense" if offense category is the same as updated offense category; "diff offense" otherwise

### 1.3.1 Code that deliberately throws error due to creating a new column in same step as row filter

In [10]:
## create new col 
## using case when logic
## and then row filter
## throws error bc doesn't
## know that charge_update exists
compare_charge_filt_q = """
select *,
CASE 
WHEN OFFENSE_CATEGORY = UPDATED_OFFENSE_CATEGORY THEN 'Same offense'
    ELSE 'Diff offense'
END as charge_update
from caseinit
where charge_update = 'Diff offense'
"""

#compare_charge_filt_d = pd.read_sql_query(compare_charge_filt_q, cnx)


### 1.3.1 Fix: instead of creating a new column and then filtering, filter directly 

# 2. Subqueries and aggregation: one dataset

Here we're going to remain focused on the `caseinit` dataset

## 2.1 Row filtering using subquery

- Write an inner query to construct the same `charge_update` column discussed above
- Then, use the outer query to filter to rows where `charge_update = 'Diff offense'` 

## 2.2 Longer example: racial disparities in who's offered diversion

-  Find the five most common offenses in the `caseinit` table for Black or White defendants (filter out other race/ethnicities)
- For those five most common offenses, find the percent of Black defendants whose cases are diverted and the percent of White defendants whose cases are diverted


### 2.2.1 Step one: write the part of the code that finds the most common offenses

### 2.2.2 Step two: add that as a subquery to filter to rows where the UPDATED_OFFENSE_CATEGORY is in one of those top 5 offenses

### 2.2.3 Step three: write the part of the code that finds proportion diverted by race/ethnicity 

### 2.2.4 Step four: combine all pieces of the code to find prop diverted for top 5 offenses

# 2.3 Activity break 1: are elderly defendants more likely to receive diversion?

- Create a new column -- `is_elderly` when pulling from the caseinit table that takes on the value of 1 if the defendant's `AGE_AT_INCIDENT` is > 65; 0 otherwise
- Use `where` to row filter to initiations where the defendant is elderly and use group by to find the count of cases diverted and not diverted (`is_in_diversion`); pull the table with those counts
- Find the proportion of cases diverted for elderly versus non-elderly defendants (mean `is_in_diversion`} by group

# 3. Subqueries and aggregations: two datasets

Here, we're going to add in an additional table `divert` that, for the initiated cases that are sent to diversion, what types of non-prison diversion programs people are sent to

## 3.1 Looking at structure of `divert` table

## 3.2 Left joining certain cols from caseinit onto diversions

### Left join with everything from divert; certain cols from caseinit

*Task*: join `caseinit` and `divert` on CASE_ID and CASE_PARTICIPANT_ID; select all columns from divert; `AGE_AT_INCIDENT` from `caseinit`

### Deliberate error- selecting RACE col from both

In [28]:
lj_dup_q = """
select divert.*,
RACE
from divert
LEFT JOIN caseinit
ON divert.CASE_ID = caseinit.CASE_ID
AND divert.CASE_PARTICIPANT_ID = caseinit.CASE_PARTICIPANT_ID
"""

In [29]:
#lj_d_dup = pd.read_sql_query(lj_dup_q, cnx)


### Fixing that error - aliasing (temporary rename) race column

### Another way to simplify: aliasing the tables in the join

*Task*: alias the `caseinit` table as `c` and the `divert` table as `d` and perform the same join/select as above

# 4. Combining agg and join

*Task*: among the cases that are diverted, for each of the charges (`UPDATED_OFFENSE_CATEGORY`) in the case initiations, find the percentage of defendants with that charge going to each `DIVERSION_PROGRAM`

## 4.1 Step one: count of offenses by diversion program

## 4.2 Step two: count of offenses total

## 4.3 Step three: combining into one query 



# 4.4 Activity break 2 

- Create a new column using the `divert` table:  `is_vetcourt` where for values of `DIVERSION_PROGRAM` = 'VC', takes on value of 1, 0 otherwise; execute this query to make sure the query for this step is correct
-  In the case initiations table, filter to (1) defendants with `UPDATED_OFFENSE_CATEGORY` is 'Narcotics'; (2) race is Black or White; and (3) is diverted; execute this query to make sure the query for this step is correct
- Combine the queries from step 1 and 2 to find, among the defendants diverted to something for narcotics offenses, the percentage of Black and percentage of white defendants sent specifically to veteran's treatment court